In [15]:
import lightning as L
import torch
from torch.utils.data import DataLoader
import os
import sys

currentdir = os.path.dirname(os.path.abspath('.'))
sys.path.insert(0, currentdir)

from lightning_gpt import data, models
from tokenizers import (
    Tokenizer,
    decoders
)

version = 8
checkpoint_dir = f'{currentdir}/temp/checkpoints/version_{version}/'
#load hparams.yaml
import yaml
with open(f'{checkpoint_dir}hparams.yaml') as file:
    hparams = yaml.load(file, Loader=yaml.FullLoader)


block_size = hparams['block_size']
batch_size = 8
num_workers = 4
model_type = hparams['model_type']
n_layer = hparams['n_layer']
n_head = hparams['n_head']
n_embd = hparams['n_embd']
learning_rate = hparams['learning_rate']

tokenizer = Tokenizer.from_file(f"{currentdir}/temp/tokenizerBPE.json")
tokenizer.decoder = decoders.ByteLevel()
test_file = f'{currentdir}/temp/val.bin'
test_dataset = data.cc_czech_Dataset(test_file, block_size,tokenizer)

test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

extra_kwargs = {}

extra_kwargs["dropout"] = 0.1

model = models.NanoGPTExport(
    vocab_size=test_dataset.vocab_size,
    block_size=test_dataset.block_size,
    model_type=model_type,
    n_layer=n_layer,
    n_head=n_head,
    n_embd=n_embd,
    weight_decay=0.1,
    learning_rate=learning_rate,
    betas=(0.9, 0.95),
    **extra_kwargs,
)

data has 612173 characters, 50000 unique.


number of parameters: 6.60M


In [16]:
checkpoint_path = f'{checkpoint_dir}checkpoints/epoch=99-step=368400.ckpt'
model = model.load_from_checkpoint(checkpoint_path)

number of parameters: 95.10M


In [17]:
# export model to torchscript
model = model.eval()
model = model.cpu()
model.to_torchscript(f'{checkpoint_dir}model.pt')

#model.save(f'{checkpoint_dir}model.pt')

RecursiveScriptModule(
  original_name=NanoGPTExport
  (nanogpt): RecursiveScriptModule(
    original_name=GPTExport
    (transformer): RecursiveScriptModule(
      original_name=ModuleDict
      (wte): RecursiveScriptModule(original_name=Embedding)
      (wpe): RecursiveScriptModule(original_name=Embedding)
      (drop): RecursiveScriptModule(original_name=Dropout)
      (h): RecursiveScriptModule(
        original_name=ModuleList
        (0): RecursiveScriptModule(
          original_name=Block
          (ln_1): RecursiveScriptModule(original_name=LayerNorm)
          (attn): RecursiveScriptModule(
            original_name=CausalSelfAttention
            (c_attn): RecursiveScriptModule(original_name=Linear)
            (c_proj): RecursiveScriptModule(original_name=Linear)
            (attn_dropout): RecursiveScriptModule(original_name=Dropout)
            (resid_dropout): RecursiveScriptModule(original_name=Dropout)
          )
          (ln_2): RecursiveScriptModule(original_name=L

In [4]:
import torch
import os
import sys

currentdir = os.path.dirname(os.path.abspath('.'))
sys.path.insert(0, currentdir)
version = 8
checkpoint_dir = f'{currentdir}/temp/checkpoints/version_{version}/'
model = torch.jit.load(f"{checkpoint_dir}model.pt")

In [5]:

context = "Prezident české republiky bude"  # Prime with something
x = test_dataset.to_tokens(context,'cpu')
y = model(x)
#print(test_dataset.from_tokens(y.tolist()[0]))

NameError: name 'test_dataset' is not defined

In [5]:
from torch.functional import F
def generate(model,idx, max_new_tokens, block_size=128, temperature=1.0, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= block_size else idx[:, -block_size:]
            # forward the model to get the logits for the index in the sequence
            logits = model(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [6]:
# make gradio interface which takes in a string as an input and outputs a number
import gradio as gr
def inference(context):
    x = test_dataset.to_tokens(context, 'cpu')
    y = generate(model, x, max_new_tokens=100, block_size=128, temperature=1.0, top_k=10)
    return test_dataset.from_tokens(y.tolist()[0])

gr.Interface(inference, "text", "text").launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [20]:

import gradio as gr
def inferencePepr(context):
    x = test_dataset.to_tokens(context, 'cpu')
    logits = model(x)
    print(x.shape,logits.shape)
    loss = F.cross_entropy(logits.view(-1, logits.size(-1))[:-1], x.view(-1)[1:], ignore_index=-1).exp()
     #loss = torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), x.view(-1), reduction='none').view(x.size(0), -1).sum(1).mean().exp()
    return f"Perplexity: {loss}"
gr.Interface(inferencePepr, "text", "text").launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


torch.Size([1, 5]) torch.Size([1, 5, 50000])
torch.Size([1, 4]) torch.Size([1, 4, 50000])
torch.Size([1, 5]) torch.Size([1, 5, 50000])
torch.Size([1, 5]) torch.Size([1, 5, 50000])
torch.Size([1, 7]) torch.Size([1, 7, 50000])
torch.Size([1, 6]) torch.Size([1, 6, 50000])
torch.Size([1, 5]) torch.Size([1, 5, 50000])
torch.Size([1, 7]) torch.Size([1, 7, 50000])
torch.Size([1, 6]) torch.Size([1, 6, 50000])
torch.Size([1, 4]) torch.Size([1, 4, 50000])


In [19]:
from torch.functional import F
perplexities = []
model = model.eval()
model = model.cpu()
for i,data in enumerate(test_loader):
    x,y = data
    logits = model(x)
    loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1), ignore_index=-1)
    perplexities.append(loss.exp().item())
    print(i)
print(f"Perplexity: {sum(perplexities)/len(perplexities)}")
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

[[197,
  2373,
  33,
  867,
  2142,
  429,
  10867,
  15484,
  15484,
  2127,
  2127,
  36785,
  36785,
  31076,
  4672,
  4672,
  29868,
  31540,
  9601,
  9601,
  40550,
  42754,
  34371,
  8463,
  13303,
  38525,
  546,
  25597,
  25597,
  39468,
  16591,
  33805,
  14723,
  24611,
  2776,
  31036,
  28202,
  23911,
  23911,
  11362,
  7753,
  44529,
  42547,
  12625,
  2480,
  25875,
  11818,
  4625,
  12687,
  38143,
  842,
  15371,
  9968,
  37434,
  13323,
  1678,
  1678,
  25954,
  43125,
  43125,
  37160,
  43747,
  44920,
  6550,
  27462,
  27462,
  27462,
  43567,
  7794,
  41934,
  41934,
  28222,
  26177,
  32461,
  7374,
  30186,
  13317,
  8981,
  45675,
  42610,
  34041,
  15749,
  15404,
  17192,
  42049,
  42049,
  176,
  42581,
  43912,
  28733,
  39813,
  45201,
  28005,
  47204,
  15554,
  29279,
  37265,
  37265,
  37265,
  39076,
  6639,
  4161,
  6568,
  6568,
  31493,
  16450,
  14121,
  32275,
  47166,
  4064,
  3521,
  15874,
  46299,
  12730,
  40644,
  6712

In [50]:
loss

tensor(9.0933, grad_fn=<NllLossBackward0>)

In [25]:
x

tensor([[ 197, 2373,   33,  867, 2142,  429]])